In [1]:
from zipfile import ZipFile
import os
from google.colab import files
! pip install tensorboardx
! pip install bayesian-optimization

ROOT = '/content'

     |████████████████████████████████| 245kB 4.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/1d/0d/3b/6b9d4477a34b3905f246ff4e7acf6aafd4cc9b77d473629b77
Successfully built bayesian-optimization


Install ngrok for tunneling and start tensorboard

In [2]:
if not os.path.exists(os.path.join(ROOT, "ngrok-stable-linux-amd64.zip")):
  !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
  !unzip ngrok-stable-linux-amd64.zip
  
LOG_DIR = os.path.join(ROOT,'saved/log/')
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 6006 &')

--2019-06-04 21:00:28--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.72.145.109, 54.236.200.27, 52.73.94.166, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.72.145.109|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16648024 (16M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  15.88M  18.4MB/s    in 0.9s    

2019-06-04 21:00:29 (18.4 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [16648024/16648024]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


Setup tensorboard

In [3]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://e92047fe.ngrok.io


Upload project as .zip file and extract it

In [4]:
file = files.upload()

zip_file = ZipFile(os.path.join(ROOT,list(file.keys())[0]))
zip_file.extractall(ROOT)
zip_file.close()

Saving Archive.zip to Archive.zip


Modify parameters and train model **manually**

In [0]:
from config import CONFIG  
from parse_config import ConfigParser
from train import main

CONFIG['arch']['args']['num_channels'] = 256
CONFIG['arch']['args']['num_stacks'] = 2
CONFIG['arch']['args']['num_blocks'] = 6
CONFIG['arch']['args']['kernel_size'] = 5

CONFIG['data_loader']['args']['data_dir']="data/XRay/Patient_0"
CONFIG['data_loader']['args']['validation_split']=0.2
CONFIG['data_loader']['args']['shuffle']=True

CONFIG['optimizer']['args']['lr'] = 0.0003

CONFIG['trainer']['epochs'] = 1000
CONFIG['trainer']['save_period'] = 20
CONFIG['trainer']['early_stop'] = 50


CONFIG['fraction_of_dataset']=1
CONFIG['sigma'] = 120
CONFIG['prediction_blur'] = 2
CONFIG['threshold'] = 0.01

main(ConfigParser(CONFIG))

/content/model/model.py:81: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (1 <= num_blocks <= 2, "invalid number of blocks [1, 2]")


StackedHourglassNet(
  (conv1): Conv2d(1, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (relu): ReLU()
  (hgs): ModuleList(
    (0): Hourglass(
      (pre_bottleneck_blocks): ModuleList(
        (0): Bottleneck(
          (conv1): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
          (conv2): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
          (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
          (relu): ReLU()
        )
        (1): Bottleneck(
          (conv1): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
          (conv2): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
          (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
          (relu): ReLU()
        )
        (2): Bottleneck(
          (conv1): Conv2d(256, 128, kernel_size=(1, 1), stride=(1, 1))
          (conv2): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
          (conv3): Conv2d(256, 256, kern

Do **automatic** Bayesian optimization

In [5]:
from bayes_opt_train import run_bayes_opt

run_bayes_opt({
    'num_channels': (6, 8),  # {64, 128, 256}
    'num_stacks': (2, 7),
    'num_blocks': (1, 7),
    'kernel_size': (1, 4),  # {3, 5, 7, 9}
    'sigma': (0.6, 5),
    'prediction_blur': (0.01, 1),
    'threshold': (0.00001, 0.2),
    'epochs': (200, 200)

}, init_points=10, n_iter=10)

/content/model/model.py:80: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (1 <= num_blocks <= 2, "invalid number of blocks [1, 2]")


|   iter    |  target   |  epochs   | kernel... | num_bl... | num_ch... | num_st... |
-------------------------------------------------------------------------------------
StackedHourglassNet(
  (conv1): Conv2d(1, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (relu): ReLU()
  (hgs): ModuleList(
    (0): Hourglass(
      (pre_bottleneck_blocks): ModuleList(
        (0): Bottleneck(
          (conv1): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
          (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv3): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
          (relu): ReLU()
        )
      )
      (intermediate_bottleneck_blocks): ModuleList(
        (0): Bottleneck(
          (conv1): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1))
          (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (conv3): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
          (relu): ReLU()
     

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:804: UserWarning: Using a target size (torch.Size([1, 1, 1507328])) that is different to the input size (torch.Size([1507328])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction)


Train Epoch: 1 [0/1 (0%)] Loss: 0.089002


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:443: UserWarning: Using a target size (torch.Size([1, 1, 1507328])) that is different to the input size (torch.Size([1507328])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


...

    epoch          : 1
    loss           : 0.027021177113056183
    percentage_correct_keypoints: 0.0
    keypoint_distance_loss: 169.57233171935775
    mse_loss       : 0.054043203592300415
    val_loss       : 0.4929101765155792
    val_percentage_correct_keypoints: 8.695652173913043
    val_keypoint_distance_loss: 197.94415730953602
    val_mse_loss   : 1.4116017818450928
Train Epoch: 2 [0/1 (0%)] Loss: 0.492910
    epoch          : 2
    loss           : 0.4929101765155792
    percentage_correct_keypoints: 8.695652173913043
    keypoint_distance_loss: 197.94415730953602
    mse_loss       : 1.4116017818450928
    val_loss       : 0.04072883352637291
    val_percentage_correct_keypoints: 4.3478260869565215
    val_keypoint_distance_loss: 155.97341395530464
    val_mse_loss   : 0.08165403455495834
Train Epoch: 3 [0/1 (0%)] Loss: 0.040729
    epoch          : 3
    loss           : 0.04072883352637291
    percentage_correct_keypoints: 4.3478260869565215
    keypoint_distance_loss: 15

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:804: UserWarning: Using a target size (torch.Size([1, 1, 1507328])) that is different to the input size (torch.Size([1507328])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.smooth_l1_loss(input, target, reduction=self.reduction)


Train Epoch: 1 [0/1 (0%)] Loss: 0.070415


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:443: UserWarning: Using a target size (torch.Size([1, 1, 1507328])) that is different to the input size (torch.Size([1507328])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


    epoch          : 1
    loss           : 0.07041539996862411
    percentage_correct_keypoints: 13.043478260869565
    keypoint_distance_loss: 149.97060127777723
    mse_loss       : 0.1419900357723236
    val_loss       : 0.0217884611338377
    val_percentage_correct_keypoints: 13.043478260869565
    val_keypoint_distance_loss: 141.59675526736203
    val_mse_loss   : 0.043580085039138794
Train Epoch: 2 [0/1 (0%)] Loss: 0.021788
    epoch          : 2
    loss           : 0.0217884611338377
    percentage_correct_keypoints: 13.043478260869565
    keypoint_distance_loss: 141.59675526736203
    mse_loss       : 0.043580085039138794
    val_loss       : 0.01165461353957653
    val_percentage_correct_keypoints: 17.391304347826086
    val_keypoint_distance_loss: 120.68637539843103
    val_mse_loss   : 0.02330923266708851
Train Epoch: 3 [0/1 (0%)] Loss: 0.011655
    epoch          : 3
    loss           : 0.01165461353957653
    percentage_correct_keypoints: 17.391304347826086
    keypoint